<a href="https://colab.research.google.com/github/carsonnj/Project-4---Store-Sales-ML/blob/main/Store_Sales_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.cluster import KMeans
from pathlib import Path
!pip install hvplot
import hvplot.pandas
!pip install holoviews
import holoviews as hv
from holoviews import opts


In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
store_sales_df = pd.read_csv("https://raw.githubusercontent.com/carsonnj/Project-4---Store-Sales-ML/main/Stores.csv")
store_sales_df.head()

In [ ]:
# Convert Store_Area from square yards to square feet
yards_to_ft = 9
store_sales_df['sq_ft'] = store_sales_df['Store_Area'] * yards_to_ft

# Display the first five rows of the dataframe
store_sales_df.head()

In [ ]:
#Divide Sales/sq. feet for sales $ per sq. ft
store_sales_df['sales_per_customer'] = store_sales_df['Store_Sales'] / store_sales_df['Daily_Customer_Count']
#Show df
store_sales_df.head()

In [ ]:
#Divide Sales/sq. feet for sales $ per sq. ft
store_sales_df['sales_per_sq_ft'] = store_sales_df['Store_Sales'] / store_sales_df['sq_ft']
#Show df
store_sales_df.head()

Clustering Model

In [ ]:
 # Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))
 # Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(store_sales_df)
    inertia.append(k_model.inertia_)
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

In [ ]:
from matplotlib import pyplot as plt
_df_6['inertia'].plot(kind='line', figsize=(8, 4), title='inertia')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)


In [ ]:
# Define the model with the lower value of k clusters
# Use a random_state of 1 to generate the model
model = KMeans(n_clusters=4, random_state=1)

# Fit the model
model.fit(store_sales_df)

# Make predictions
k_lower = model.predict(store_sales_df)

# Create a copy of the DataFrame
store_sales_cluster_df = store_sales_df.copy()

# Add a class column with the labels to the spread_df_predictions DataFrame
store_sales_cluster_df['cluster_number'] = k_lower

#Print prediction
store_sales_cluster_df.head()

In [ ]:
#Start Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
# Create a SparkSession object
spark = SparkSession.builder.getOrCreate()
# Create a DataFrame from the existing store_sales_cluster_df variable
store_sales_cluster_df = spark.createDataFrame(store_sales_cluster_df)
#create temp table to use spark.sql to understand cluster
store_sales_cluster_df.createOrReplaceTempView("cluster")
#group by cluster and look at average sales, sq. ft, and sales per sq. ft
store_sales_cluster_df.groupBy("cluster_number").agg(avg("Store_Sales"), avg("Sq_Ft"), avg("Sales_Per_Sq_Ft"), avg("Items_Available"), avg("Daily_Customer_Count"), avg("sales_per_customer")).show()

In [ ]:
store_sales_df.describe()

Feature Engineering to Optimize models

In [ ]:
# copy DF
store_sales_copy_df = store_sales_df.copy()
#Remove stores with less than 100 daily customerss
store_sales_copy_df = store_sales_copy_df[store_sales_copy_df['Daily_Customer_Count'] >= 250]
#Add Customers/Items
store_sales_copy_df['customer_per_item'] = store_sales_copy_df['Daily_Customer_Count'] /store_sales_copy_df['Items_Available']
#Add Items/sq. ft
store_sales_copy_df['items_per_sq_ft'] = store_sales_copy_df['Items_Available']/store_sales_copy_df['sq_ft']
#Add Customers/sq. ft
store_sales_copy_df['customer_per_sq_ft'] = store_sales_copy_df['Daily_Customer_Count'] /store_sales_copy_df['sq_ft']
store_sales_copy_df.describe()


Evaluate Data distribution

In [ ]:
#distribution of items avaliable
store_sales_copy_df['Items_Available'].plot(kind='hist', bins=20, title='Items_Available')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
#distribution of store sales
store_sales_copy_df['Store_Sales'].plot(kind='hist', bins=20, title='Store_Sales')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
# Scatter Plot with items and store sales to see if there is a linear correlation
plt.scatter(store_sales_copy_df['Items_Available'], store_sales_copy_df['Store_Sales'])

# Set the title and axis labels
plt.title('Sales vs. Items Available')
plt.xlabel('Items Available')
plt.ylabel('Sales')

# Show the plot
plt.show()


In [ ]:
# Scatter Plot with sq. ft and sales to see if there is a linear correlation
plt.scatter(store_sales_df['sq_ft'], store_sales_df['Store_Sales'])

# Set the title and axis labels
plt.title('Sales vs. Square Feet')
plt.xlabel('Square Feet')
plt.ylabel('Sales')

# Show the plot
plt.show()


In [ ]:
# Scatter Plot with items and sq. ft to see if there is a linear correlation
plt.scatter(store_sales_copy_df['Items_Available'], store_sales_copy_df['sq_ft'])

# Set the title and axis labels
plt.title('Items Available vs. Square Feet')
plt.xlabel('Items Available')
plt.ylabel('Square Feet')

# Show the plot
plt.show()


In [ ]:
# Create correlations df

Correlation_df = store_sales_df[['sq_ft', 'Items_Available', 'Daily_Customer_Count', 'Store_Sales']]


In [ ]:
#import seaboarn as sns

!pip install seaborn
import seaborn as sns


In [ ]:
sns.set_theme(style="ticks",palette='deep')
sns.pairplot(Correlation_df)

Linear Regression Model


In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
# Extract features (X) and target variable (y)
X = store_sales_copy_df[['Items_Available', 'sq_ft', 'Daily_Customer_Count','customer_per_item','items_per_sq_ft','customer_per_sq_ft']]
y = store_sales_copy_df['Store_Sales']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

In [ ]:
import numpy as np
# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
#Training score
print(f"Training Score: {model.score(X_train_scaled, y_train)}")
# MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test_scaled)))
print("RMSE:", rmse)
# R2
r2 = metrics.r2_score(y_test, y_pred)
print(f"R2 score: {r2}")
# Visualize predicted vs. actual values
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Store Sales')
plt.ylabel('Predicted Store Sales')
plt.title('Actual vs. Predicted Store Sales')
plt.show()

In [ ]:
# Feature Importance
feature_importance = model.coef_
print('Feature Importance:', feature_importance)

Random Forest


In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
# Extract features (X) and target variable (y)
X = store_sales_copy_df[['Items_Available', 'sq_ft', 'Daily_Customer_Count','customer_per_item','items_per_sq_ft','customer_per_sq_ft']]
y = store_sales_copy_df['Store_Sales']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Model Evaluation
y_pred = rf_model.predict(X_test)
# Evaluate the model using metrics like Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Feature Importance
feature_importance = rf_model.feature_importances_
print('Feature Importance:', feature_importance)

In [ ]:
# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)
#Training score
print(f"Training Score: {model.score(X_train_scaled, y_train)}")
# MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test_scaled)))
print("RMSE:", rmse)
# R2
r2 = metrics.r2_score(y_test, y_pred)
print(f"R2 score: {r2}")
# Visualize predicted vs. actual values
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Store Sales')
plt.ylabel('Predicted Store Sales')
plt.title('Actual vs. Predicted Store Sales')
plt.show()

XGBoost Model

In [ ]:
# Models
from xgboost.spark import SparkXGBRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [ ]:
from sklearn.model_selection import train_test_split
# Extract features (X) and target variable (y)
X = store_sales_copy_df[['Items_Available', 'sq_ft', 'Daily_Customer_Count','customer_per_item','items_per_sq_ft','customer_per_sq_ft']]
y = store_sales_copy_df['Store_Sales']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
#xgboost model
xgbr = XGBRegressor(random_state = 116)
xgbr.fit(X_train_scaled, y_train)
xgbr_pred =xgbr.predict(X_test_scaled)

In [ ]:
import numpy as np
mse = mean_squared_error(y_test, xgbr_pred)
rmse = np.sqrt(mean_squared_error(y_test, xgbr_pred))
score = xgbr.score(X_train, y_train)
r2 =r2_score(y_test,  xgbr_pred)

In [ ]:
print("Training score: ", score)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (rmse))
print("R_score : %.2f"%(r2))
# Visualize predicted vs. actual values
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Store Sales')
plt.ylabel('Predicted Store Sales')
plt.title('Actual vs. Predicted Store Sales')
plt.show()

KNN Model

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
# Extract features (X) and target variable (y)
X = store_sales_copy_df[['Items_Available', 'sq_ft', 'Daily_Customer_Count']]
y = store_sales_copy_df['Store_Sales']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=3, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))

In [ ]:
# Visualize training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
#MSE
mse = mean_squared_error(y_test, model.predict(X_test_scaled))
print("MSE:", mse)
# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test_scaled)))
print("RMSE:", rmse)
# R2
r2 = metrics.r2_score(y_test, y_pred)
print(f"R2 score: {r2}")
# Visualize predicted vs. actual values
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Store Sales')
plt.ylabel('Predicted Store Sales')
plt.title('Actual vs. Predicted Store Sales')
plt.show()